In [7]:
from vllm import LLM, EngineArgs, PoolingParams

In [2]:
prompts = [
    "Follow the white rabbit.",  # English
    "Sigue al conejo blanco.",  # Spanish
    "Suis le lapin blanc.",  # French
    "跟着白兔走。",  # Chinese
    "اتبع الأرنب الأبيض.",  # Arabic
    "Folge dem weißen Kaninchen.",  # German
]

In [3]:
model = LLM(
    model="jinaai/jina-embeddings-v3",
    task="embed",
    trust_remote_code=True,
)

config.json: 0.00B [00:00, ?B/s]

configuration_xlm_roberta.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/jinaai/xlm-roberta-flash-implementation:
- configuration_xlm_roberta.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modules.json:   0%|          | 0.00/378 [00:00<?, ?B/s]

INFO 09-24 19:56:49 [config.py:484] Found sentence-transformers modules configuration.


...

INFO 09-24 19:56:59 [rocm.py:233] Using ROCmFlashAttention backend.
INFO 09-24 19:56:59 [parallel_state.py:1076] rank 0 in world size 1 is assigned as DP rank 0, PP rank 0, TP rank 0, EP rank 0
INFO 09-24 19:56:59 [model_runner.py:1171] Starting to load model jinaai/jina-embeddings-v3...
INFO 09-24 19:57:00 [weight_utils.py:292] Using model weights format ['*.safetensors']


model.safetensors:   0%|          | 0.00/1.14G [00:00<?, ?B/s]

INFO 09-24 19:57:02 [weight_utils.py:308] Time spent downloading weights for jinaai/jina-embeddings-v3: 2.445407 seconds
INFO 09-24 19:57:02 [weight_utils.py:345] No model.safetensors.index.json found in remote.


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 09-24 19:57:10 [default_loader.py:272] Loading weights took 7.89 seconds
INFO 09-24 19:57:10 [model_runner.py:1203] Model loading took 4.1230 GiB and 10.823411 seconds


#### General

In [4]:
outputs = model.embed(prompts)

Adding requests:   0%|          | 0/6 [00:00<?, ?it/s]

Processed prompts:   0% 0/6 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

In [5]:
print("\nGenerated Outputs:")
print("-" * 60)
for prompt, output in zip(prompts, outputs):
    embeds = output.outputs.embedding
    embeds_trimmed = (
        (str(embeds[:16])[:-1] + ", ...]") if len(embeds) > 16 else embeds
    )
    print(
        f"Prompt: {prompt!r} \n"
        f"Embeddings for text matching: {embeds_trimmed} "
        f"(size={len(embeds)})"
    )
    print("-" * 60)


Generated Outputs:
------------------------------------------------------------
Prompt: 'Follow the white rabbit.' 
Embeddings for text matching: [-0.14213059842586517, -0.05029097571969032, 0.01353879552334547, 0.04582967981696129, 0.08003295212984085, 0.03540872409939766, -0.0008887738804332912, 0.05841369181871414, -0.04777868092060089, -0.03277672827243805, -0.07327341288328171, 0.061782196164131165, -0.08115953952074051, 0.06381005793809891, -0.08413373678922653, -0.026829740032553673, ...] (size=1024)
------------------------------------------------------------
Prompt: 'Sigue al conejo blanco.' 
Embeddings for text matching: [-0.05010094866156578, -0.047637730836868286, -0.045011844485998154, -0.025765040889382362, 0.13794027268886566, -0.027189088985323906, -0.002701407764106989, 0.052564166486263275, -0.08811818063259125, 0.006689615547657013, -0.02919844165444374, 0.001295513822697103, -0.038563322275877, 0.08588734269142151, -0.12074421346187592, 0.06291375309228897, ...] (s

#### With PoolingParams

In [8]:
outputs = model.embed(prompts, pooling_params=PoolingParams(dimensions=32))

Adding requests:   0%|          | 0/6 [00:00<?, ?it/s]

Processed prompts:   0% 0/6 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

In [9]:
print("\nGenerated Outputs:")
print("-" * 60)
for prompt, output in zip(prompts, outputs):
    embeds = output.outputs.embedding
    embeds_trimmed = (
        (str(embeds[:16])[:-1] + ", ...]") if len(embeds) > 16 else embeds
    )
    print(f"Prompt: {prompt!r} \nEmbeddings: {embeds_trimmed} (size={len(embeds)})")
    print("-" * 60)


Generated Outputs:
------------------------------------------------------------
Prompt: 'Follow the white rabbit.' 
Embeddings: [-0.38497915863990784, -0.13621963560581207, 0.036671582609415054, 0.12413563579320908, 0.21677963435649872, 0.09590911865234375, -0.0024073594249784946, 0.1582210659980774, -0.12941475212574005, -0.08878002315759659, -0.19847054779529572, 0.1673450917005539, -0.21983115375041962, 0.17283782362937927, -0.22788715362548828, -0.07267183065414429, ...] (size=32)
------------------------------------------------------------
Prompt: 'Sigue al conejo blanco.' 
Embeddings: [-0.13791604340076447, -0.13113538920879364, -0.1239069476723671, -0.07092504948377609, 0.3797169029712677, -0.07484512031078339, -0.007436335552483797, 0.1446966975927353, -0.2425684779882431, 0.018414925783872604, -0.08037639409303665, 0.003566242754459381, -0.10615569353103638, 0.23642750084400177, -0.3323802053928375, 0.17318665981292725, ...] (size=32)
-----------------------------------------